In [1]:
## import library 
import os
import pandas as pd
import db_dtypes
import pickle

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "trial_bigq.json"

In [3]:
project_id = 'dtidsUS'
dataset_id = 'capstone'
table_id = 'data_saudi_used_cars'
region = 'us-central1'
bucket_name = 'modul4'
blob_name = 'puji/data_saudi_used_cars.csv'

In [4]:
client = bigquery.Client(project='dtidsus')

# Upload Dataset

In [5]:
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket('modul4')
    data_capstone = bucket.blob('puji/data_saudi_used_cars.csv')
    data_capstone.upload_from_filename('data_saudi_used_cars.csv')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [6]:
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")

In [7]:
df = query_job.result().to_dataframe()

C:\Users\nde\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


# Upload the Model 

In [8]:
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket('modul4')
    blob_model = bucket.blob('puji/model/model.pkl') #blob_model = bucket.blob('ilham/model/model.pkl')
    blob_model.upload_from_filename('model.pkl') #blob_model.upload_from_filename('model.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [9]:
from google.cloud import aiplatform

In [10]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "dev_trial.json"

In [11]:
aiplatform.init(project='dtidsus', location='us-central1')

model = aiplatform.Model.upload(
    display_name='puji_model_001', #'jaya_model_000'
    artifact_uri="gs://modul4/puji/model/", #"gs://modul4/ilham/model/"
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

Creating Model
Create Model backing LRO: projects/41965541199/locations/us-central1/models/7455521166623506432/operations/5169168547200368640
Model created. Resource name: projects/41965541199/locations/us-central1/models/7455521166623506432@1
To use this Model in another session:
model = aiplatform.Model('projects/41965541199/locations/us-central1/models/7455521166623506432@1')


# Create Endpoint (?)

In [12]:
endpoint = aiplatform.Endpoint.create(
    display_name="puji-endpoint-001", #"jaya-endpoint-000"
    project='dtidsus',
    location='us-central1',
)

Creating Endpoint
Create Endpoint backing LRO: projects/41965541199/locations/us-central1/endpoints/897387305730310144/operations/6326030701481164800
Endpoint created. Resource name: projects/41965541199/locations/us-central1/endpoints/897387305730310144
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/41965541199/locations/us-central1/endpoints/897387305730310144')


In [13]:
min_replica_count: int = 1
max_replica_count: int = 1

In [14]:
# Deploy model to the endpoint
endpoint.deploy( 
    model=model,
    deployed_model_display_name='puji_model_001', #'jaya_model_000'
    machine_type='e2-standard-2',
    min_replica_count=min_replica_count,
    max_replica_count=max_replica_count,
    sync=True, 
)

Deploying Model projects/41965541199/locations/us-central1/models/7455521166623506432 to Endpoint : projects/41965541199/locations/us-central1/endpoints/897387305730310144
Deploy Endpoint model backing LRO: projects/41965541199/locations/us-central1/endpoints/897387305730310144/operations/5166353797433262080


FailedPrecondition: 400 Model server exited unexpectedly. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=41965541199&resource=aiplatform.googleapis.com%2FDeploymentResourcePool&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FDeploymentResourcePool%22%0Aresource.labels.deployment_resource_pool_id%3D%22internal_6333569606036226048%22%0Aresource.labels.location%3D%22us-central1%22. 9: Model server exited unexpectedly. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=41965541199&resource=aiplatform.googleapis.com%2FDeploymentResourcePool&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FDeploymentResourcePool%22%0Aresource.labels.deployment_resource_pool_id%3D%22internal_6333569606036226048%22%0Aresource.labels.location%3D%22us-central1%22.

# Using Endpoint

In [ ]:
# predict your data with online prediction here 
PROJECT_ID = 'dtidsus'
ENDPOINT_ID = "projects/41965541199/locations/us-central1/puji-endpoint-000/2241324766407426048"
REGION = 'us-central1'

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint = aiplatform.Endpoint(ENDPOINT_ID)
prediction = endpoint.predict(instances=[[50000, 1, 3, 1, 23, 131895, 132154, 129237]])

print("PREDICTION:", prediction)

In [ ]:
# Manual File

In [ ]:
try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob('puji/model/model_catboost.pkl')
    blob_model.download_to_filename('model_catboost_new.pkl')

    print ("download model succeeded")
except:
    raise TypeError("An exception occurred")

In [ ]:
model_load = pickle.load(open('model_catboost_new.pkl','rb'))

In [ ]:
y_predict = model_load.predict(X_test)

In [ ]:
y_predict